In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# TimeSformer

In [2]:
import numpy as np

with open("/content/drive/Shareddrives/CV_Action_Recognition_Project/Dataset/UCF/npy_files/" + "TimeSformer_rgb_and_flow.npy", 'rb') as f:
    data_oc = np.load(f, allow_pickle=True)
    data_uc = np.load(f, allow_pickle=True)
    data_lc = np.load(f, allow_pickle=True)
    data_of = np.load(f, allow_pickle=True)
    data_uf = np.load(f, allow_pickle=True)
    data_lf = np.load(f, allow_pickle=True)
    label = np.load(f, allow_pickle=True)


In [3]:
labels = label

In [4]:
import keras
#labels = keras.utils.to_categorical(labels)

In [5]:
labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12])

In [6]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model

import autokeras as ak

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for train_index, test_index in kf.split(data_oc, labels):
  data_oc_train, data_oc_test = data_oc[train_index], data_oc[test_index]
  data_uc_train, data_uc_test = data_uc[train_index], data_uc[test_index]
  data_lc_train, data_lc_test = data_lc[train_index], data_lc[test_index]
  data_of_train, data_of_test = data_of[train_index], data_of[test_index]
  data_uf_train, data_uf_test = data_uf[train_index], data_uf[test_index]
  data_lf_train, data_lf_test = data_lf[train_index], data_lf[test_index]

  label_train, label_test = labels[train_index], labels[test_index]

  break

In [8]:
label_train

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [9]:
label_test

array([ 0,  0,  0,  1,  2,  2,  3,  4,  4,  5,  5,  7,  7,  8,  8,  8,  9,
        9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12])

In [10]:
label_train = keras.utils.to_categorical(label_train)
label_test = keras.utils.to_categorical(label_test)


In [11]:
label_test.shape

(28, 13)

### stage 1

In [12]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, objective = 'val_accuracy', max_trials=150) 

clf.fit(
    data_oc_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(data_oc_test, label_test)[1]
print(test_accuracy)
model = clf.export_model()

model.summary()

Epoch 1/3
4/4 [==============================] - 1s 7ms/step - loss: 2.9217 - accuracy: 0.0625
Epoch 2/3
4/4 [==============================] - 0s 6ms/step - loss: 1.2101 - accuracy: 0.7500
Epoch 3/3
4/4 [==============================] - 0s 6ms/step - loss: 0.7441 - accuracy: 0.8661


1/1 [==============================] - 0s 190ms/step - loss: 0.8866 - accuracy: 0.7857
0.7857142686843872
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768)]             0         
                                                                 
 normalization (Normalizatio  (None, 768)              1537      
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               196864    
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                4112      
                                                                 
 re_lu_1 (ReLU)      

In [27]:
from keras import layers

def get_one_stream_model(input_data):
  input_shape = (None, 768)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(16, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  #x = layers.Dropout(0.3)(x)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model

In [45]:
def one_stream_test_and_return (data_train, data_test):
  epochs = 200
  model = get_one_stream_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-6].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [50]:
data_oc_test.shape

(28, 768)

In [51]:
label_train.shape

(112, 13)

In [52]:
label_test.shape

(28, 13)

In [51]:
v_oc_train, v_oc_test = one_stream_test_and_return(data_oc_train, data_oc_test)

Epoch 1/200
4/4 [==============================] - 1s 69ms/step - loss: 2.8234 - accuracy: 0.0893 - val_loss: 2.6808 - val_accuracy: 0.0357
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5209 - accuracy: 0.1250 - val_loss: 2.4805 - val_accuracy: 0.0714
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 2.2836 - accuracy: 0.1429 - val_loss: 2.3108 - val_accuracy: 0.1429
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.0778 - accuracy: 0.2232 - val_loss: 2.1646 - val_accuracy: 0.1786
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 1.9140 - accuracy: 0.3125 - val_loss: 2.0428 - val_accuracy: 0.3571
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 1.7739 - accuracy: 0.4018 - val_loss: 1.9429 - val_accuracy: 0.3929
Epoch 7/200
4/4 [==============================] - 0s 14ms/step - loss: 1.6497 - accuracy: 0.4732 - val_loss: 1.8575 - val_accuracy: 0.4286
Epoch 8/200
4/4 [===

In [49]:
v_oc_test.shape

(28, 256)

In [48]:
v_uc_train, v_uc_test = one_stream_test_and_return(data_uc_train, data_uc_test)

Epoch 1/200
4/4 [==============================] - 1s 68ms/step - loss: 2.6843 - accuracy: 0.0893 - val_loss: 2.5830 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.3662 - accuracy: 0.1786 - val_loss: 2.3803 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 13ms/step - loss: 2.1402 - accuracy: 0.3214 - val_loss: 2.2244 - val_accuracy: 0.2857
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 1.9656 - accuracy: 0.3839 - val_loss: 2.1129 - val_accuracy: 0.2857
Epoch 5/200
4/4 [==============================] - 0s 21ms/step - loss: 1.8361 - accuracy: 0.4286 - val_loss: 2.0278 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 1.7239 - accuracy: 0.4821 - val_loss: 1.9577 - val_accuracy: 0.3214
Epoch 7/200
4/4 [==============================] - 0s 14ms/step - loss: 1.6274 - accuracy: 0.5268 - val_loss: 1.8959 - val_accuracy: 0.3929
Epoch 8/200
4/4 [===

In [26]:
v_uc_train.shape

(112, 13)

In [50]:
v_lc_train, v_lc_test = one_stream_test_and_return(data_lc_train, data_lc_test)

Epoch 1/200
4/4 [==============================] - 1s 73ms/step - loss: 2.8153 - accuracy: 0.1250 - val_loss: 2.6375 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 17ms/step - loss: 2.5025 - accuracy: 0.1607 - val_loss: 2.4314 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 20ms/step - loss: 2.2901 - accuracy: 0.1964 - val_loss: 2.2608 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.1151 - accuracy: 0.2411 - val_loss: 2.1296 - val_accuracy: 0.2143
Epoch 5/200
4/4 [==============================] - 0s 14ms/step - loss: 1.9759 - accuracy: 0.2946 - val_loss: 2.0308 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 19ms/step - loss: 1.8631 - accuracy: 0.3125 - val_loss: 1.9504 - val_accuracy: 0.2857
Epoch 7/200
4/4 [==============================] - 0s 16ms/step - loss: 1.7661 - accuracy: 0.3571 - val_loss: 1.8855 - val_accuracy: 0.2857
Epoch 8/200
4/4 [===

In [52]:
v_of_train, v_of_test = one_stream_test_and_return(data_of_train, data_of_test)

Epoch 1/200
4/4 [==============================] - 1s 72ms/step - loss: 2.8509 - accuracy: 0.1250 - val_loss: 2.9160 - val_accuracy: 0.1786
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5535 - accuracy: 0.1964 - val_loss: 2.7964 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 18ms/step - loss: 2.3334 - accuracy: 0.2232 - val_loss: 2.6817 - val_accuracy: 0.2143
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.1381 - accuracy: 0.3214 - val_loss: 2.5879 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 14ms/step - loss: 1.9733 - accuracy: 0.3839 - val_loss: 2.5047 - val_accuracy: 0.3571
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 1.8300 - accuracy: 0.4196 - val_loss: 2.4355 - val_accuracy: 0.3571
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 1.7047 - accuracy: 0.4643 - val_loss: 2.3743 - val_accuracy: 0.3929
Epoch 8/200
4/4 [===

In [53]:
v_uf_train, v_uf_test = one_stream_test_and_return(data_uf_train, data_uf_test)

Epoch 1/200
4/4 [==============================] - 1s 77ms/step - loss: 2.6944 - accuracy: 0.1339 - val_loss: 2.6056 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 15ms/step - loss: 2.4292 - accuracy: 0.1786 - val_loss: 2.4869 - val_accuracy: 0.1429
Epoch 3/200
4/4 [==============================] - 0s 16ms/step - loss: 2.2306 - accuracy: 0.2589 - val_loss: 2.3906 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 2.0556 - accuracy: 0.3214 - val_loss: 2.3074 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 1.8976 - accuracy: 0.4375 - val_loss: 2.2362 - val_accuracy: 0.3214
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 1.7602 - accuracy: 0.4911 - val_loss: 2.1617 - val_accuracy: 0.3571
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 1.6346 - accuracy: 0.5625 - val_loss: 2.0925 - val_accuracy: 0.3571
Epoch 8/200
4/4 [===

In [54]:
v_lf_train, v_lf_test = one_stream_test_and_return(data_lf_train, data_lf_test)

Epoch 1/200
4/4 [==============================] - 1s 68ms/step - loss: 2.7802 - accuracy: 0.0625 - val_loss: 2.7480 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 2.5815 - accuracy: 0.0982 - val_loss: 2.6792 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 2.4377 - accuracy: 0.1161 - val_loss: 2.6201 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.3120 - accuracy: 0.1339 - val_loss: 2.5651 - val_accuracy: 0.2143
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2001 - accuracy: 0.2143 - val_loss: 2.5155 - val_accuracy: 0.2143
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 2.1077 - accuracy: 0.2500 - val_loss: 2.4706 - val_accuracy: 0.2857
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 2.0264 - accuracy: 0.2768 - val_loss: 2.4275 - val_accuracy: 0.3214
Epoch 8/200
4/4 [===

### stage 2

In [55]:
v_o_train = np.concatenate([v_oc_train, v_of_train], axis=1)
v_o_test = np.concatenate([v_oc_test, v_of_test], axis=1)

v_u_train = np.concatenate([v_uc_train, v_uf_train], axis=1)
v_u_test = np.concatenate([v_uc_test, v_uf_test], axis=1)

v_l_train = np.concatenate([v_lc_train, v_lf_train], axis=1)
v_l_test = np.concatenate([v_lc_test, v_lf_test], axis=1)


In [56]:
v_u_test.shape

(28, 512)

In [79]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=50, objective = 'val_accuracy', project_name="stage 2") 

clf.fit(
    v_u_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(v_u_test, label_test)[1]
model = clf.export_model()

model.summary()

1/1 [==============================] - 0s 185ms/step - loss: 0.5777 - accuracy: 0.8929
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 normalization (Normalizatio  (None, 512)              1025      
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                        

In [85]:
def get_stage_2_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.BatchNormalization(axis=1)(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dropout(0.5)(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [86]:
def stage_2_test_and_return (data_train, data_test):
  epochs = 200
  model = get_stage_2_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-4].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [87]:
_, _ = stage_2_test_and_return(v_o_train, v_o_test)

Epoch 1/200
4/4 [==============================] - 1s 112ms/step - loss: 3.5328 - accuracy: 0.1161 - val_loss: 3.1373 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 20ms/step - loss: 3.1298 - accuracy: 0.1607 - val_loss: 2.9275 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 25ms/step - loss: 2.8766 - accuracy: 0.1339 - val_loss: 2.7294 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 25ms/step - loss: 2.7004 - accuracy: 0.1786 - val_loss: 2.5516 - val_accuracy: 0.1786
Epoch 5/200
4/4 [==============================] - 0s 25ms/step - loss: 2.4661 - accuracy: 0.2321 - val_loss: 2.3842 - val_accuracy: 0.2143
Epoch 6/200
4/4 [==============================] - 0s 27ms/step - loss: 2.4234 - accuracy: 0.2857 - val_loss: 2.2244 - val_accuracy: 0.2857
Epoch 7/200
4/4 [==============================] - 0s 25ms/step - loss: 2.2396 - accuracy: 0.3036 - val_loss: 2.0779 - val_accuracy: 0.3571
Epoch 8/200
4/4 [==

In [88]:
v2_u_train, v2_u_test = stage_2_test_and_return(v_u_train, v_u_test)

Epoch 1/200
4/4 [==============================] - 1s 71ms/step - loss: 3.3355 - accuracy: 0.0536 - val_loss: 2.7368 - val_accuracy: 0.0357
Epoch 2/200
4/4 [==============================] - 0s 20ms/step - loss: 3.1819 - accuracy: 0.1429 - val_loss: 2.5305 - val_accuracy: 0.0714
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 3.1216 - accuracy: 0.0893 - val_loss: 2.3388 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 2.7437 - accuracy: 0.1696 - val_loss: 2.1647 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 14ms/step - loss: 2.6019 - accuracy: 0.1339 - val_loss: 2.0067 - val_accuracy: 0.3214
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 2.4163 - accuracy: 0.2589 - val_loss: 1.8623 - val_accuracy: 0.4286
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 2.2644 - accuracy: 0.2500 - val_loss: 1.7307 - val_accuracy: 0.4286
Epoch 8/200
4/4 [===

In [89]:
v2_l_train, v2_l_test = stage_2_test_and_return(v_l_train, v_l_test)

Epoch 1/200
4/4 [==============================] - 1s 79ms/step - loss: 3.3893 - accuracy: 0.0893 - val_loss: 3.1112 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 3.1995 - accuracy: 0.0804 - val_loss: 2.8541 - val_accuracy: 0.0000e+00
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 2.9670 - accuracy: 0.0804 - val_loss: 2.6108 - val_accuracy: 0.0357
Epoch 4/200
4/4 [==============================] - 0s 20ms/step - loss: 2.8337 - accuracy: 0.0804 - val_loss: 2.3817 - val_accuracy: 0.0357
Epoch 5/200
4/4 [==============================] - 0s 15ms/step - loss: 2.3869 - accuracy: 0.2411 - val_loss: 2.1716 - val_accuracy: 0.1071
Epoch 6/200
4/4 [==============================] - 0s 16ms/step - loss: 2.2119 - accuracy: 0.2679 - val_loss: 1.9767 - val_accuracy: 0.2143
Epoch 7/200
4/4 [==============================] - 0s 17ms/step - loss: 2.1845 - accuracy: 0.2679 - val_loss: 1.8030 - val_accuracy: 0.2500
Epoch 8/200


### stage 3

In [90]:
v_2_train = np.concatenate([v2_u_train, v2_l_train], axis=1)
v_2_test = np.concatenate([v2_u_test, v2_l_test], axis=1)

In [99]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=150, objective = 'val_accuracy', project_name="stage 4") 

clf.fit(
    v_2_train,
    label_train,
    validation_data = [v_2_test, label_test]
)

test_accuracy = clf.evaluate(v_2_test, label_test)[1]
model = clf.export_model()

model.summary()

Trial 85 Complete [00h 00m 08s]
val_accuracy: 0.9285714030265808

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 08s


1/1 [==============================] - 0s 162ms/step - loss: 0.1663 - accuracy: 1.0000
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 re_lu_1 (ReLU)              (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 13)                845       
                                        

In [96]:
def get_stage_3_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  outputs = layers.Dense(32, activation='linear')(input)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(64, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [100]:
def stage_3_test(data_train, data_test):
  epochs = 200
  model = get_stage_3_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


In [101]:
stage_3_test(v_2_train, v_2_test)

Epoch 1/200
4/4 [==============================] - 1s 65ms/step - loss: 2.4968 - accuracy: 0.1339 - val_loss: 2.2456 - val_accuracy: 0.2500
Epoch 2/200
4/4 [==============================] - 0s 15ms/step - loss: 2.0349 - accuracy: 0.4018 - val_loss: 1.9677 - val_accuracy: 0.4643
Epoch 3/200
4/4 [==============================] - 0s 17ms/step - loss: 1.6838 - accuracy: 0.5714 - val_loss: 1.7128 - val_accuracy: 0.5714
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 1.3421 - accuracy: 0.7054 - val_loss: 1.4638 - val_accuracy: 0.6071
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 1.0280 - accuracy: 0.7768 - val_loss: 1.2282 - val_accuracy: 0.6786
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 0.7633 - accuracy: 0.8482 - val_loss: 1.0246 - val_accuracy: 0.7857
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 0.5568 - accuracy: 0.9375 - val_loss: 0.8428 - val_accuracy: 0.7857
Epoch 8/200
4/4 [===

# R3D_18_rgb_and_flow

In [102]:
import numpy as np

with open("/content/drive/Shareddrives/CV_Action_Recognition_Project/Dataset/UCF/npy_files/" + "R3D_18_rgb_and_flow.npy", 'rb') as f:
    data_oc = np.load(f, allow_pickle=True)
    data_uc = np.load(f, allow_pickle=True)
    data_lc = np.load(f, allow_pickle=True)
    data_of = np.load(f, allow_pickle=True)
    data_uf = np.load(f, allow_pickle=True)
    data_lf = np.load(f, allow_pickle=True)
    label = np.load(f, allow_pickle=True)


In [103]:
labels = label

In [104]:
import keras
#labels = keras.utils.to_categorical(labels)

In [105]:
labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12])

In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model

import autokeras as ak

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for train_index, test_index in kf.split(data_oc, labels):
  data_oc_train, data_oc_test = data_oc[train_index], data_oc[test_index]
  data_uc_train, data_uc_test = data_uc[train_index], data_uc[test_index]
  data_lc_train, data_lc_test = data_lc[train_index], data_lc[test_index]
  data_of_train, data_of_test = data_of[train_index], data_of[test_index]
  data_uf_train, data_uf_test = data_uf[train_index], data_uf[test_index]
  data_lf_train, data_lf_test = data_lf[train_index], data_lf[test_index]

  label_train, label_test = labels[train_index], labels[test_index]

  break

In [107]:
label_train

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [108]:
label_test

array([ 0,  0,  0,  1,  2,  2,  3,  4,  4,  5,  5,  7,  7,  8,  8,  8,  9,
        9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12])

In [109]:
label_train = keras.utils.to_categorical(label_train)
label_test = keras.utils.to_categorical(label_test)


In [110]:
label_test.shape

(28, 13)

### stage 1

In [111]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, objective = 'val_accuracy', max_trials=150) 

clf.fit(
    data_oc_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(data_oc_test, label_test)[1]
print(test_accuracy)
model = clf.export_model()

model.summary()

ValueError: ignored

In [112]:
from keras import layers

def get_one_stream_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(16, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  #x = layers.Dropout(0.3)(x)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model

In [113]:
def one_stream_test_and_return (data_train, data_test):
  epochs = 200
  model = get_one_stream_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-6].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [114]:
data_oc_test.shape

(28, 512)

In [115]:
label_train.shape

(112, 13)

In [116]:
label_test.shape

(28, 13)

In [117]:
v_oc_train, v_oc_test = one_stream_test_and_return(data_oc_train, data_oc_test)

Epoch 1/200
4/4 [==============================] - 1s 74ms/step - loss: 2.7728 - accuracy: 0.0714 - val_loss: 2.7836 - val_accuracy: 0.0357
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5996 - accuracy: 0.1339 - val_loss: 2.6778 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 2.4539 - accuracy: 0.1875 - val_loss: 2.5813 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.3274 - accuracy: 0.2232 - val_loss: 2.4887 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 15ms/step - loss: 2.2058 - accuracy: 0.3036 - val_loss: 2.4043 - val_accuracy: 0.2500
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 2.0891 - accuracy: 0.3571 - val_loss: 2.3284 - val_accuracy: 0.2500
Epoch 7/200
4/4 [==============================] - 0s 20ms/step - loss: 1.9878 - accuracy: 0.3839 - val_loss: 2.2552 - val_accuracy: 0.2500
Epoch 8/200
4/4 [===

In [118]:
v_oc_test.shape

(28, 256)

In [119]:
v_uc_train, v_uc_test = one_stream_test_and_return(data_uc_train, data_uc_test)

Epoch 1/200
4/4 [==============================] - 1s 72ms/step - loss: 2.7692 - accuracy: 0.0536 - val_loss: 2.6921 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 2.5979 - accuracy: 0.0625 - val_loss: 2.6120 - val_accuracy: 0.0000e+00
Epoch 3/200
4/4 [==============================] - 0s 21ms/step - loss: 2.4620 - accuracy: 0.0893 - val_loss: 2.5393 - val_accuracy: 0.0000e+00
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.3373 - accuracy: 0.1161 - val_loss: 2.4732 - val_accuracy: 0.0000e+00
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2252 - accuracy: 0.1786 - val_loss: 2.4111 - val_accuracy: 0.1071
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 2.1224 - accuracy: 0.2500 - val_loss: 2.3543 - val_accuracy: 0.1786
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.0295 - accuracy: 0.3125 - val_loss: 2.3052 - val_accuracy: 0.2143
Epoc

In [120]:
v_uc_train.shape

(112, 256)

In [121]:
v_lc_train, v_lc_test = one_stream_test_and_return(data_lc_train, data_lc_test)

Epoch 1/200
4/4 [==============================] - 1s 107ms/step - loss: 2.9220 - accuracy: 0.0982 - val_loss: 2.7655 - val_accuracy: 0.0357
Epoch 2/200
4/4 [==============================] - 0s 18ms/step - loss: 2.7173 - accuracy: 0.1518 - val_loss: 2.6396 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 23ms/step - loss: 2.5511 - accuracy: 0.1786 - val_loss: 2.5301 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 18ms/step - loss: 2.4105 - accuracy: 0.1875 - val_loss: 2.4356 - val_accuracy: 0.1786
Epoch 5/200
4/4 [==============================] - 0s 23ms/step - loss: 2.2910 - accuracy: 0.2143 - val_loss: 2.3589 - val_accuracy: 0.2143
Epoch 6/200
4/4 [==============================] - 0s 19ms/step - loss: 2.1824 - accuracy: 0.2500 - val_loss: 2.2921 - val_accuracy: 0.2500
Epoch 7/200
4/4 [==============================] - 0s 22ms/step - loss: 2.0864 - accuracy: 0.2857 - val_loss: 2.2338 - val_accuracy: 0.2857
Epoch 8/200
4/4 [==

In [122]:
v_of_train, v_of_test = one_stream_test_and_return(data_of_train, data_of_test)

Epoch 1/200
4/4 [==============================] - 1s 65ms/step - loss: 2.7539 - accuracy: 0.0625 - val_loss: 2.9050 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.6011 - accuracy: 0.0714 - val_loss: 2.8605 - val_accuracy: 0.0714
Epoch 3/200
4/4 [==============================] - 0s 12ms/step - loss: 2.4819 - accuracy: 0.1161 - val_loss: 2.8252 - val_accuracy: 0.0357
Epoch 4/200
4/4 [==============================] - 0s 18ms/step - loss: 2.3697 - accuracy: 0.1339 - val_loss: 2.7958 - val_accuracy: 0.0357
Epoch 5/200
4/4 [==============================] - 0s 15ms/step - loss: 2.2772 - accuracy: 0.1964 - val_loss: 2.7709 - val_accuracy: 0.0357
Epoch 6/200
4/4 [==============================] - 0s 20ms/step - loss: 2.1925 - accuracy: 0.2589 - val_loss: 2.7509 - val_accuracy: 0.0714
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.1174 - accuracy: 0.3125 - val_loss: 2.7363 - val_accuracy: 0.1071
Epoch 8/200
4/4 

In [123]:
v_uf_train, v_uf_test = one_stream_test_and_return(data_uf_train, data_uf_test)

Epoch 1/200
4/4 [==============================] - 1s 77ms/step - loss: 2.6639 - accuracy: 0.0982 - val_loss: 2.5998 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5057 - accuracy: 0.1518 - val_loss: 2.5606 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 2.3828 - accuracy: 0.1786 - val_loss: 2.5262 - val_accuracy: 0.2143
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2760 - accuracy: 0.2232 - val_loss: 2.4943 - val_accuracy: 0.1786
Epoch 5/200
4/4 [==============================] - 0s 19ms/step - loss: 2.1819 - accuracy: 0.2768 - val_loss: 2.4637 - val_accuracy: 0.1786
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 2.0929 - accuracy: 0.3036 - val_loss: 2.4360 - val_accuracy: 0.2143
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.0155 - accuracy: 0.3750 - val_loss: 2.4099 - val_accuracy: 0.2143
Epoch 8/200
4/4 [===

In [124]:
v_lf_train, v_lf_test = one_stream_test_and_return(data_lf_train, data_lf_test)

Epoch 1/200
4/4 [==============================] - 1s 66ms/step - loss: 2.9244 - accuracy: 0.0893 - val_loss: 2.8549 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 2.7435 - accuracy: 0.0893 - val_loss: 2.7928 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 18ms/step - loss: 2.6081 - accuracy: 0.0893 - val_loss: 2.7412 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 2.4858 - accuracy: 0.1071 - val_loss: 2.6945 - val_accuracy: 0.1071
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 2.3838 - accuracy: 0.1250 - val_loss: 2.6539 - val_accuracy: 0.1071
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 2.2916 - accuracy: 0.1607 - val_loss: 2.6135 - val_accuracy: 0.1071
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2101 - accuracy: 0.1607 - val_loss: 2.5793 - val_accuracy: 0.1429
Epoch 8/200
4/4 [===

### stage 2

In [125]:
v_o_train = np.concatenate([v_oc_train, v_of_train], axis=1)
v_o_test = np.concatenate([v_oc_test, v_of_test], axis=1)

v_u_train = np.concatenate([v_uc_train, v_uf_train], axis=1)
v_u_test = np.concatenate([v_uc_test, v_uf_test], axis=1)

v_l_train = np.concatenate([v_lc_train, v_lf_train], axis=1)
v_l_test = np.concatenate([v_lc_test, v_lf_test], axis=1)


In [126]:
v_u_test.shape

(28, 512)

In [127]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=50, objective = 'val_accuracy', project_name="stage 2") 

clf.fit(
    v_u_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(v_u_test, label_test)[1]
model = clf.export_model()

model.summary()

1/1 [==============================] - 0s 177ms/step - loss: 2.7896 - accuracy: 0.1071
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 normalization (Normalizatio  (None, 512)              1025      
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                        

In [128]:
def get_stage_2_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.BatchNormalization(axis=1)(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dropout(0.5)(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [129]:
def stage_2_test_and_return (data_train, data_test):
  epochs = 200
  model = get_stage_2_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-4].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [130]:
_, _ = stage_2_test_and_return(v_o_train, v_o_test)

Epoch 1/200
4/4 [==============================] - 1s 80ms/step - loss: 3.5272 - accuracy: 0.0536 - val_loss: 3.0518 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 3.2498 - accuracy: 0.1518 - val_loss: 2.8843 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 16ms/step - loss: 2.9907 - accuracy: 0.1071 - val_loss: 2.7272 - val_accuracy: 0.1786
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.7483 - accuracy: 0.1607 - val_loss: 2.5780 - val_accuracy: 0.2857
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 2.7242 - accuracy: 0.1964 - val_loss: 2.4430 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5678 - accuracy: 0.2143 - val_loss: 2.3168 - val_accuracy: 0.2857
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.4190 - accuracy: 0.1875 - val_loss: 2.1972 - val_accuracy: 0.3571
Epoch 8/200
4/4 [===

In [131]:
v2_u_train, v2_u_test = stage_2_test_and_return(v_u_train, v_u_test)

Epoch 1/200
4/4 [==============================] - 1s 70ms/step - loss: 3.5114 - accuracy: 0.0625 - val_loss: 3.1753 - val_accuracy: 0.0357
Epoch 2/200
4/4 [==============================] - 0s 19ms/step - loss: 3.3816 - accuracy: 0.1161 - val_loss: 3.0369 - val_accuracy: 0.0357
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 3.0814 - accuracy: 0.1250 - val_loss: 2.9085 - val_accuracy: 0.0357
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.9841 - accuracy: 0.1964 - val_loss: 2.7876 - val_accuracy: 0.0714
Epoch 5/200
4/4 [==============================] - 0s 19ms/step - loss: 2.6567 - accuracy: 0.1696 - val_loss: 2.6734 - val_accuracy: 0.1786
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 2.4600 - accuracy: 0.2321 - val_loss: 2.5650 - val_accuracy: 0.2143
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5028 - accuracy: 0.1696 - val_loss: 2.4617 - val_accuracy: 0.2500
Epoch 8/200
4/4 [===

In [132]:
v2_l_train, v2_l_test = stage_2_test_and_return(v_l_train, v_l_test)

Epoch 1/200
4/4 [==============================] - 1s 100ms/step - loss: 3.1973 - accuracy: 0.0625 - val_loss: 3.1527 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 29ms/step - loss: 3.0116 - accuracy: 0.1339 - val_loss: 2.9906 - val_accuracy: 0.0357
Epoch 3/200
4/4 [==============================] - 0s 25ms/step - loss: 2.9486 - accuracy: 0.1161 - val_loss: 2.8424 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 25ms/step - loss: 2.6048 - accuracy: 0.2232 - val_loss: 2.7035 - val_accuracy: 0.1429
Epoch 5/200
4/4 [==============================] - 0s 25ms/step - loss: 2.5469 - accuracy: 0.1518 - val_loss: 2.5761 - val_accuracy: 0.2143
Epoch 6/200
4/4 [==============================] - 0s 18ms/step - loss: 2.3972 - accuracy: 0.2411 - val_loss: 2.4535 - val_accuracy: 0.2500
Epoch 7/200
4/4 [==============================] - 0s 25ms/step - loss: 2.4003 - accuracy: 0.2232 - val_loss: 2.3377 - val_accuracy: 0.3214
Epoch 8/200
4/4

### stage 3

In [133]:
v_2_train = np.concatenate([v2_u_train, v2_l_train], axis=1)
v_2_test = np.concatenate([v2_u_test, v2_l_test], axis=1)

In [137]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=150, objective = 'val_accuracy', project_name="stage 3 CNN") 

clf.fit(
    v_2_train,
    label_train,
    validation_data = [v_2_test, label_test]
)

test_accuracy = clf.evaluate(v_2_test, label_test)[1]
model = clf.export_model()

model.summary()

Trial 60 Complete [00h 00m 05s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 09m 50s

Search: Running Trial #61

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
256               |128               |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate
1024              |1024              |structured_data_block_1/dense_block_1/units_2

Epoch 1/1000


KeyboardInterrupt: ignored

In [141]:
def get_stage_3_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  outputs = layers.Dense(32, activation='linear')(input)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [142]:
def stage_3_test(data_train, data_test):
  epochs = 200
  model = get_stage_3_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


In [143]:
stage_3_test(v_2_train, v_2_test)

Epoch 1/200
4/4 [==============================] - 1s 73ms/step - loss: 2.5448 - accuracy: 0.1429 - val_loss: 2.4126 - val_accuracy: 0.2500
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 2.1662 - accuracy: 0.4911 - val_loss: 2.2156 - val_accuracy: 0.4643
Epoch 3/200
4/4 [==============================] - 0s 20ms/step - loss: 1.8001 - accuracy: 0.7054 - val_loss: 1.9983 - val_accuracy: 0.5000
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 1.4104 - accuracy: 0.7500 - val_loss: 1.7573 - val_accuracy: 0.5357
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 1.0071 - accuracy: 0.8571 - val_loss: 1.4906 - val_accuracy: 0.5714
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 0.6579 - accuracy: 0.9018 - val_loss: 1.2542 - val_accuracy: 0.6071
Epoch 7/200
4/4 [==============================] - 0s 12ms/step - loss: 0.4110 - accuracy: 0.9196 - val_loss: 1.0940 - val_accuracy: 0.5357
Epoch 8/200
4/4 [===

# Swin_T_rgb_and_flow

In [144]:
import numpy as np

with open("/content/drive/Shareddrives/CV_Action_Recognition_Project/Dataset/UCF/npy_files/" + "Swin_T_rgb_and_flow.npy", 'rb') as f:
    data_oc = np.load(f, allow_pickle=True)
    data_uc = np.load(f, allow_pickle=True)
    data_lc = np.load(f, allow_pickle=True)
    data_of = np.load(f, allow_pickle=True)
    data_uf = np.load(f, allow_pickle=True)
    data_lf = np.load(f, allow_pickle=True)
    label = np.load(f, allow_pickle=True)


In [145]:
labels = label

In [146]:
import keras
#labels = keras.utils.to_categorical(labels)

In [147]:
labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12])

In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [148]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model

import autokeras as ak

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for train_index, test_index in kf.split(data_oc, labels):
  data_oc_train, data_oc_test = data_oc[train_index], data_oc[test_index]
  data_uc_train, data_uc_test = data_uc[train_index], data_uc[test_index]
  data_lc_train, data_lc_test = data_lc[train_index], data_lc[test_index]
  data_of_train, data_of_test = data_of[train_index], data_of[test_index]
  data_uf_train, data_uf_test = data_uf[train_index], data_uf[test_index]
  data_lf_train, data_lf_test = data_lf[train_index], data_lf[test_index]

  label_train, label_test = labels[train_index], labels[test_index]

  break

In [149]:
label_train

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  5,  5,  5,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [150]:
label_test

array([ 0,  0,  0,  1,  2,  2,  3,  4,  4,  5,  5,  7,  7,  8,  8,  8,  9,
        9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12])

In [151]:
label_train = keras.utils.to_categorical(label_train)
label_test = keras.utils.to_categorical(label_test)


In [152]:
label_test.shape

(28, 13)

### stage 1

In [ ]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, objective = 'val_accuracy', max_trials=150) 

clf.fit(
    data_oc_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(data_oc_test, label_test)[1]
print(test_accuracy)
model = clf.export_model()

model.summary()

ValueError: ignored

In [153]:
from keras import layers

def get_one_stream_model(input_data):
  input_shape = (None, 768)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dense(16, activation='linear')(outputs)
  outputs = layers.ReLU()(outputs)
  #x = layers.Dropout(0.3)(x)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model

In [154]:
def one_stream_test_and_return (data_train, data_test):
  epochs = 200
  model = get_one_stream_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-6].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [155]:
data_oc_test.shape

(28, 768)

In [156]:
label_train.shape

(112, 13)

In [157]:
label_test.shape

(28, 13)

In [159]:
v_oc_train, v_oc_test = one_stream_test_and_return(data_oc_train, data_oc_test)

Epoch 1/200
4/4 [==============================] - 1s 102ms/step - loss: 2.8152 - accuracy: 0.1071 - val_loss: 3.0121 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 20ms/step - loss: 2.4815 - accuracy: 0.1696 - val_loss: 2.7698 - val_accuracy: 0.1429
Epoch 3/200
4/4 [==============================] - 0s 20ms/step - loss: 2.2293 - accuracy: 0.2411 - val_loss: 2.5611 - val_accuracy: 0.2500
Epoch 4/200
4/4 [==============================] - 0s 25ms/step - loss: 2.0071 - accuracy: 0.3571 - val_loss: 2.3899 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 21ms/step - loss: 1.8324 - accuracy: 0.4464 - val_loss: 2.2424 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 25ms/step - loss: 1.6849 - accuracy: 0.5268 - val_loss: 2.1158 - val_accuracy: 0.3571
Epoch 7/200
4/4 [==============================] - 0s 19ms/step - loss: 1.5608 - accuracy: 0.6161 - val_loss: 2.0037 - val_accuracy: 0.4643
Epoch 8/200
4/4 [==

In [160]:
v_oc_test.shape

(28, 256)

In [164]:
v_uc_train, v_uc_test = one_stream_test_and_return(data_uc_train, data_uc_test)

Epoch 1/200
4/4 [==============================] - 1s 72ms/step - loss: 2.7829 - accuracy: 0.1339 - val_loss: 2.6776 - val_accuracy: 0.1786
Epoch 2/200
4/4 [==============================] - 0s 17ms/step - loss: 2.5414 - accuracy: 0.1429 - val_loss: 2.5391 - val_accuracy: 0.2143
Epoch 3/200
4/4 [==============================] - 0s 21ms/step - loss: 2.3377 - accuracy: 0.2054 - val_loss: 2.4347 - val_accuracy: 0.2500
Epoch 4/200
4/4 [==============================] - 0s 20ms/step - loss: 2.1822 - accuracy: 0.2589 - val_loss: 2.3523 - val_accuracy: 0.2500
Epoch 5/200
4/4 [==============================] - 0s 18ms/step - loss: 2.0525 - accuracy: 0.2946 - val_loss: 2.2772 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 16ms/step - loss: 1.9364 - accuracy: 0.3304 - val_loss: 2.2109 - val_accuracy: 0.2857
Epoch 7/200
4/4 [==============================] - 0s 19ms/step - loss: 1.8391 - accuracy: 0.3661 - val_loss: 2.1538 - val_accuracy: 0.2857
Epoch 8/200
4/4 [===

In [162]:
v_uc_train.shape

(112, 256)

In [163]:
v_lc_train, v_lc_test = one_stream_test_and_return(data_lc_train, data_lc_test)

Epoch 1/200
4/4 [==============================] - 1s 72ms/step - loss: 2.6964 - accuracy: 0.1071 - val_loss: 2.4835 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 16ms/step - loss: 2.4199 - accuracy: 0.1696 - val_loss: 2.3353 - val_accuracy: 0.1429
Epoch 3/200
4/4 [==============================] - 0s 23ms/step - loss: 2.2230 - accuracy: 0.2679 - val_loss: 2.2029 - val_accuracy: 0.1429
Epoch 4/200
4/4 [==============================] - 0s 15ms/step - loss: 2.0673 - accuracy: 0.3125 - val_loss: 2.0820 - val_accuracy: 0.2143
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 1.9165 - accuracy: 0.3750 - val_loss: 1.9765 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 1.7957 - accuracy: 0.4911 - val_loss: 1.8858 - val_accuracy: 0.3929
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 1.6919 - accuracy: 0.5357 - val_loss: 1.8055 - val_accuracy: 0.4286
Epoch 8/200
4/4 [===

In [165]:
v_of_train, v_of_test = one_stream_test_and_return(data_of_train, data_of_test)

Epoch 1/200
4/4 [==============================] - 1s 70ms/step - loss: 2.9871 - accuracy: 0.0357 - val_loss: 3.1486 - val_accuracy: 0.0714
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.7407 - accuracy: 0.1071 - val_loss: 3.0229 - val_accuracy: 0.0714
Epoch 3/200
4/4 [==============================] - 0s 13ms/step - loss: 2.5493 - accuracy: 0.1429 - val_loss: 2.9170 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 19ms/step - loss: 2.3889 - accuracy: 0.1696 - val_loss: 2.8258 - val_accuracy: 0.1071
Epoch 5/200
4/4 [==============================] - 0s 15ms/step - loss: 2.2379 - accuracy: 0.2679 - val_loss: 2.7522 - val_accuracy: 0.1071
Epoch 6/200
4/4 [==============================] - 0s 20ms/step - loss: 2.1204 - accuracy: 0.3304 - val_loss: 2.6912 - val_accuracy: 0.1071
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 2.0038 - accuracy: 0.3839 - val_loss: 2.6444 - val_accuracy: 0.2143
Epoch 8/200
4/4 [===

In [166]:
v_uf_train, v_uf_test = one_stream_test_and_return(data_uf_train, data_uf_test)

Epoch 1/200
4/4 [==============================] - 1s 67ms/step - loss: 2.8387 - accuracy: 0.0804 - val_loss: 2.8382 - val_accuracy: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 2.5884 - accuracy: 0.1250 - val_loss: 2.7877 - val_accuracy: 0.0357
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 2.4025 - accuracy: 0.1786 - val_loss: 2.7465 - val_accuracy: 0.0357
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2443 - accuracy: 0.2500 - val_loss: 2.7095 - val_accuracy: 0.0714
Epoch 5/200
4/4 [==============================] - 0s 19ms/step - loss: 2.0966 - accuracy: 0.3125 - val_loss: 2.6749 - val_accuracy: 0.1071
Epoch 6/200
4/4 [==============================] - 0s 13ms/step - loss: 1.9723 - accuracy: 0.3571 - val_loss: 2.6483 - val_accuracy: 0.1071
Epoch 7/200
4/4 [==============================] - 0s 21ms/step - loss: 1.8628 - accuracy: 0.4107 - val_loss: 2.6237 - val_accuracy: 0.1071
Epoch 8/200
4/4 

In [167]:
v_lf_train, v_lf_test = one_stream_test_and_return(data_lf_train, data_lf_test)

Epoch 1/200
4/4 [==============================] - 1s 70ms/step - loss: 3.0807 - accuracy: 0.0714 - val_loss: 2.8352 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 2.8097 - accuracy: 0.1071 - val_loss: 2.7557 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 2.6030 - accuracy: 0.1429 - val_loss: 2.6849 - val_accuracy: 0.1071
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 2.4262 - accuracy: 0.1786 - val_loss: 2.6225 - val_accuracy: 0.1071
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 2.2768 - accuracy: 0.2054 - val_loss: 2.5690 - val_accuracy: 0.1429
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 2.1517 - accuracy: 0.2321 - val_loss: 2.5245 - val_accuracy: 0.1786
Epoch 7/200
4/4 [==============================] - 0s 18ms/step - loss: 2.0349 - accuracy: 0.2589 - val_loss: 2.4863 - val_accuracy: 0.2143
Epoch 8/200
4/4 [===

### stage 2

In [168]:
v_o_train = np.concatenate([v_oc_train, v_of_train], axis=1)
v_o_test = np.concatenate([v_oc_test, v_of_test], axis=1)

v_u_train = np.concatenate([v_uc_train, v_uf_train], axis=1)
v_u_test = np.concatenate([v_uc_test, v_uf_test], axis=1)

v_l_train = np.concatenate([v_lc_train, v_lf_train], axis=1)
v_l_test = np.concatenate([v_lc_test, v_lf_test], axis=1)


In [169]:
v_u_test.shape

(28, 512)

In [170]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=50, objective = 'val_accuracy', project_name="stage 2") 

clf.fit(
    v_u_train,
    label_train,
    validation_data = [v_u_test, label_test]
)

test_accuracy = clf.evaluate(v_u_test, label_test)[1]
model = clf.export_model()

model.summary()

KeyboardInterrupt: ignored

In [171]:
def get_stage_2_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  norm_layer = layers.Normalization(axis=-1)
  norm_layer.adapt(input_data)
  outputs = norm_layer(input)
  outputs = layers.Dense(256, activation='linear')(outputs)
  outputs = layers.BatchNormalization(axis=1)(outputs)
  outputs = layers.ReLU()(outputs)
  outputs = layers.Dropout(0.5)(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [172]:
def stage_2_test_and_return (data_train, data_test):
  epochs = 200
  model = get_stage_2_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


  model_ex = Model(inputs=model.input, outputs=model.layers[-4].output)
  v_train = model_ex.predict(data_train)
  v_test = model_ex.predict(data_test)
  return v_train, v_test

In [173]:
_, _ = stage_2_test_and_return(v_o_train, v_o_test)

Epoch 1/200
4/4 [==============================] - 1s 120ms/step - loss: 3.2190 - accuracy: 0.0357 - val_loss: 2.7821 - val_accuracy: 0.1071
Epoch 2/200
4/4 [==============================] - 0s 20ms/step - loss: 3.0172 - accuracy: 0.1429 - val_loss: 2.5828 - val_accuracy: 0.1071
Epoch 3/200
4/4 [==============================] - 0s 25ms/step - loss: 2.7664 - accuracy: 0.1786 - val_loss: 2.3978 - val_accuracy: 0.1429
Epoch 4/200
4/4 [==============================] - 0s 24ms/step - loss: 2.6320 - accuracy: 0.1518 - val_loss: 2.2265 - val_accuracy: 0.2143
Epoch 5/200
4/4 [==============================] - 0s 23ms/step - loss: 2.4167 - accuracy: 0.2054 - val_loss: 2.0672 - val_accuracy: 0.2143
Epoch 6/200
4/4 [==============================] - 0s 22ms/step - loss: 2.1990 - accuracy: 0.2500 - val_loss: 1.9177 - val_accuracy: 0.3571
Epoch 7/200
4/4 [==============================] - 0s 23ms/step - loss: 2.1219 - accuracy: 0.3304 - val_loss: 1.7832 - val_accuracy: 0.3929
Epoch 8/200
4/4 [==

In [174]:
v2_u_train, v2_u_test = stage_2_test_and_return(v_u_train, v_u_test)

Epoch 1/200
4/4 [==============================] - 1s 80ms/step - loss: 3.2883 - accuracy: 0.0982 - val_loss: 2.6330 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 3.1801 - accuracy: 0.0893 - val_loss: 2.4800 - val_accuracy: 0.2143
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 3.0471 - accuracy: 0.1339 - val_loss: 2.3368 - val_accuracy: 0.2500
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 2.7222 - accuracy: 0.1696 - val_loss: 2.2067 - val_accuracy: 0.3214
Epoch 5/200
4/4 [==============================] - 0s 14ms/step - loss: 2.7371 - accuracy: 0.1964 - val_loss: 2.0856 - val_accuracy: 0.3214
Epoch 6/200
4/4 [==============================] - 0s 14ms/step - loss: 2.4923 - accuracy: 0.2411 - val_loss: 1.9741 - val_accuracy: 0.3571
Epoch 7/200
4/4 [==============================] - 0s 15ms/step - loss: 2.3229 - accuracy: 0.2857 - val_loss: 1.8779 - val_accuracy: 0.4286
Epoch 8/200
4/4 [===

In [176]:
v2_l_train, v2_l_test = stage_2_test_and_return(v_l_train, v_l_test)

Epoch 1/200
4/4 [==============================] - 1s 117ms/step - loss: 3.2480 - accuracy: 0.0804 - val_loss: 2.6067 - val_accuracy: 0.1429
Epoch 2/200
4/4 [==============================] - 0s 19ms/step - loss: 2.9938 - accuracy: 0.0804 - val_loss: 2.4395 - val_accuracy: 0.2143
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 2.9356 - accuracy: 0.1339 - val_loss: 2.2827 - val_accuracy: 0.2143
Epoch 4/200
4/4 [==============================] - 0s 20ms/step - loss: 2.6480 - accuracy: 0.1964 - val_loss: 2.1390 - val_accuracy: 0.3214
Epoch 5/200
4/4 [==============================] - 0s 26ms/step - loss: 2.5115 - accuracy: 0.1875 - val_loss: 2.0072 - val_accuracy: 0.3214
Epoch 6/200
4/4 [==============================] - 0s 21ms/step - loss: 2.2628 - accuracy: 0.2500 - val_loss: 1.8844 - val_accuracy: 0.3214
Epoch 7/200
4/4 [==============================] - 0s 22ms/step - loss: 2.0485 - accuracy: 0.3214 - val_loss: 1.7763 - val_accuracy: 0.3571
Epoch 8/200
4/4 [==

### stage 3

In [177]:
v_2_train = np.concatenate([v2_u_train, v2_l_train], axis=1)
v_2_test = np.concatenate([v2_u_test, v2_l_test], axis=1)

In [179]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=150, objective = 'val_accuracy', project_name="stage 3 swin") 

clf.fit(
    v_2_train,
    label_train,
    validation_data = [v_2_test, label_test]
)

test_accuracy = clf.evaluate(v_2_test, label_test)[1]
model = clf.export_model()

model.summary()

Trial 16 Complete [00h 00m 07s]
val_accuracy: 0.8928571343421936

Best val_accuracy So Far: 0.8928571343421936
Total elapsed time: 00h 05m 23s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |32                |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
512               |512               |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam              |adam              |optimizer
0.01              |0.001             |learning_rate
32                |None              |structured_data_block_1/dense_block_1/units_2

Epoch 1/1000
4/4 [====

KeyboardInterrupt: ignored

In [183]:
def get_stage_3_model(input_data):
  input_shape = (None, 512)
  input = layers.Input(shape=(input_shape[1:]))

  outputs = layers.Dense(32, activation='linear')(input)
  outputs = layers.Dense(512, activation='linear')(outputs)
  outputs = layers.Dense(13, activation='linear')(outputs)
  outputs = layers.Softmax(axis=-1, name = "classification_head")(outputs)

  model = keras.Model(input, outputs)
  return model


In [184]:
def stage_3_test(data_train, data_test):
  epochs = 200
  model = get_stage_3_model(data_train)
  model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.001), 
              metrics = ['accuracy'])

  history = model.fit(data_train, label_train,
                    epochs = epochs, 
                    validation_data = (data_test, label_test))

  print(f"Test accuracy: {round(history.history['val_accuracy'][-1] * 100, 2)}%")


In [185]:
stage_3_test(v_2_train, v_2_test)

Epoch 1/200
4/4 [==============================] - 1s 74ms/step - loss: 2.3389 - accuracy: 0.2500 - val_loss: 1.7284 - val_accuracy: 0.5357
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 1.3013 - accuracy: 0.7589 - val_loss: 1.2501 - val_accuracy: 0.5714
Epoch 3/200
4/4 [==============================] - 0s 12ms/step - loss: 0.6876 - accuracy: 0.9554 - val_loss: 0.9442 - val_accuracy: 0.7500
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 0.3343 - accuracy: 0.9911 - val_loss: 0.7444 - val_accuracy: 0.8214
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 0.1477 - accuracy: 1.0000 - val_loss: 0.6333 - val_accuracy: 0.8571
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 0.0642 - accuracy: 1.0000 - val_loss: 0.5773 - val_accuracy: 0.7857
Epoch 7/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0316 - accuracy: 1.0000 - val_loss: 0.5460 - val_accuracy: 0.7857
Epoch 8/200
4/4 [===